In [23]:
import os
import librosa
import re
import subprocess
import unicodedata
import soundfile as sf
import subprocess
from concurrent.futures import ThreadPoolExecutor

In [ ]:
import os
print("Logical CPU cores available:", os.cpu_count())

Logical CPU cores available: 12


In [10]:
# === File Setup ===
mp3_path = r"C:\Users\Johnh\OneDrive\Desktop\MSDS\3 - Spring 2025\Deep Learning\Project\multimodal-playlist-generation\data\song_audio_files\example.mp3"
trimmed_dir = r"C:\Users\Johnh\OneDrive\Desktop\MSDS\3 - Spring 2025\Deep Learning\Project\multimodal-playlist-generation\data\song_audio_files\trimmed"
essentia_out = r"C:\Users\Johnh\OneDrive\Desktop\MSDS\3 - Spring 2025\Deep Learning\Project\multimodal-playlist-generation\data\essentia_results"


filename = os.path.basename(mp3_path).replace('.mp3', '.wav')
trimmed_path = os.path.join(trimmed_dir, filename)

In [11]:
def run_essentia(mp3_path):
    if mp3_path is None:
        return

    filename = os.path.basename(mp3_path)
    json_filename = filename.replace(".wav", ".json")

    # Full Docker volume paths (safe for Windows)
    input_mount = f"{os.path.abspath(trimmed_dir)}:/input"
    output_mount = f"{os.path.abspath(essentia_out)}:/output"

    cmd = [
        "docker", "run", "--rm",
        "-v", input_mount,
        "-v", output_mount,
        "mtgupf/essentia",
        "essentia_streaming_extractor_music",
        f"/input/{filename}",
        f"/output/{json_filename}"
    ]

    print("Running Docker command:")
    print(" ".join(cmd))  # See exactly what's being run

    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        print(f"[DONE] {filename}")
    except subprocess.CalledProcessError as e:
        print(f"[ESSENTIA FAIL] {filename}")
        print("stdout:", e.stdout)
        print("stderr:", e.stderr)

In [20]:
# List of WAVs to process
all_trimmed_files = [os.path.join(trimmed_dir, f) for f in os.listdir(trimmed_dir) if f.endswith(".wav")]
print('No. of files:', len(all_trimmed_files))

No. of files: 1000


In [ ]:
# Run with N workers
with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(run_essentia, all_trimmed_files)

In [25]:
# === Strong filename sanitizer ===
def sanitize(filename):
    filename = ''.join(c for c in filename if c.isascii() and c.isprintable())
    filename = unicodedata.normalize('NFKD', filename)
    filename = re.sub(r'[^\w\s\.-]', '', filename)
    filename = filename.replace(' ', '_')
    return filename

# === Step 1: Get missing JSONs and rename dirty files ===
clean_wavs_to_run = []

for wav in os.listdir(trimmed_dir):
    if not wav.endswith(".wav"):
        continue

    json_name = wav.replace(".wav", ".json")
    if json_name in os.listdir(essentia_out):
        continue  # already processed

    original_path = os.path.join(trimmed_dir, wav)
    clean_name = sanitize(wav)
    clean_path = os.path.join(trimmed_dir, clean_name)

    if clean_name != wav:
        if not os.path.exists(clean_path):
            os.rename(original_path, clean_path)
        print(f"[RENAMED] {wav} -> {clean_name}")
        wav = clean_name

    clean_wavs_to_run.append(wav)

print(f"{len(clean_wavs_to_run)} file(s) to process...")

1 file(s) to process...


In [26]:
clean_wavs_to_run

['1880_-_ROS_-_toxic_till_the_end_official_audio.wav']

In [27]:
# === Step 3: Run in parallel ===
with ThreadPoolExecutor(max_workers=8) as executor:
    executor.map(run_essentia, clean_wavs_to_run)

Running Docker command:
docker run --rm -v C:\Users\Johnh\OneDrive\Desktop\MSDS\3 - Spring 2025\Deep Learning\Project\multimodal-playlist-generation\data\song_audio_files\trimmed:/input -v C:\Users\Johnh\OneDrive\Desktop\MSDS\3 - Spring 2025\Deep Learning\Project\multimodal-playlist-generation\data\essentia_results:/output mtgupf/essentia essentia_streaming_extractor_music /input/1880_-_ROS_-_toxic_till_the_end_official_audio.wav /output/1880_-_ROS_-_toxic_till_the_end_official_audio.json
[ESSENTIA FAIL] 1880_-_ROS_-_toxic_till_the_end_official_audio.wav
stdout: 
stderr: [   INFO   ] MusicExtractor: Read metadata
[   INFO   ] MusicExtractor: Compute md5 audio hash, codec, length, and EBU 128 loudness
[   INFO   ] MusicExtractor: Replay gain
File looks like a completely silent file... Aborting...

